# Polymer Property Predictions



In [ ]:
import math, numpy as np, torch
from torch import nn
from torch.optim import AdamW, RMSprop
from torch.amp import GradScaler, autocast
from copy import deepcopy
from torch_geometric.nn import global_mean_pool, global_max_pool, GlobalAttention
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
from torch import nn
from torch_geometric.nn import GINEConv
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch.utils.data import Dataset
from torch_geometric.data import Data
import torch, numpy as np
import sys
sys.path.append('../..')
from src.data.polymer_dataset import LMDBDataset
from typing import Dict, Any, Tuple, NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import numpy as np
import os
import inspect
# Import the new libraries for LightGBM and XGBoost
import lightgbm as lgb
import xgboost as xgb
from dataclasses import dataclass
from typing import Optional, Tuple
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import numpy as np
from typing import Optional, Tuple
from tqdm.auto import tqdm
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as rdmd, DataStructs
import numpy as np, torch
from typing import List
from torch.utils.data import Dataset

In [ ]:
# general 
import pandas as pd
import numpy as np
from tqdm import tqdm
import ace_tools_open as tools
import optuna
import optuna.visualization as vis
import pickle
import joblib
import os 

# plotting 
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Module, Sequential, Dropout
from torch.utils.data import Subset
import torch.optim as optim
# PyTorch Geometric
from torch_geometric.nn import GINEConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from transformers import get_cosine_schedule_with_warmup

# OGB dataset 
from ogb.lsc import PygPCQM4Mv2Dataset, PCQM4Mv2Dataset
from ogb.utils import smiles2graph
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

# RDKit
# from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem

# ChemML
from chemml.chem import Molecule, RDKitFingerprint, CoulombMatrix, tensorise_molecules
from chemml.models import MLP, NeuralGraphHidden, NeuralGraphOutput
from chemml.utils import regression_metrics

# SKlearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("CUDA available:", tf.test.is_built_with_gpu_support())
print(tf.config.list_physical_devices('GPU'))
# list all GPUs
gpus = tf.config.list_physical_devices('GPU')

# check compute capability if GPU available
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Device: {gpu.name}")
        print(f"Compute Capability: {details.get('compute_capability')}")
else:
    print("No GPU found.")

In [ ]:
CWD = os.getcwd() 
PROJECT_ROOT = os.path.join(CWD, '..', '..')
PROJECT_ROOT = os.path.abspath(PROJECT_ROOT)

if os.path.exists('/kaggle'):
    DATA_ROOT = '/kaggle/input/neurips-open-polymer-prediction-2025'
    CHUNK_DIR = '/kaggle/working/processed_chunks'  
    BACKBONE_PATH = '/kaggle/input/polymer/hlgap-gnn3d-transformer-pcqm4mv2-v1.pt'
else:
    DATA_ROOT = os.path.join(PROJECT_ROOT, 'data')
    CHUNK_DIR = os.path.join(DATA_ROOT, 'processed_chunks')
    BACKBONE_PATH = os.path.join(PROJECT_ROOT, 'hlgap-gnn3d-transformer-pcqm4mv2-v1.pt')


TRAIN_LMDB = os.path.join(CHUNK_DIR, 'polymer_train3d_dist.lmdb')
TEST_LMDB = os.path.join(CHUNK_DIR, 'polymer_test3d_dist.lmdb')

# Create LMDBs if they don't exist
if not os.path.exists(TRAIN_LMDB) or not os.path.exists(TEST_LMDB):
    print('Building LMDBs...')
    os.makedirs(CHUNK_DIR, exist_ok=True)
    build_script_path = os.path.join(PROJECT_ROOT, 'scripts', 'data_preprocessing', 'build_lmdb.py')

    if not os.path.exists(build_script_path):
        print(f"ERROR: LMDB building script not found at {build_script_path}. Please check file location.")
    else:
        !python {build_script_path} train
        !python {build_script_path} test
        print('LMDB creation complete.')
else:
    print('LMDBs already exist.')

In [ ]:
# LMDB+CSV wiring (augmented-aware)
label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx = {k:i for i,k in enumerate(label_cols)}
AUG_KEY_MULT = 1000 # must match the builder

# 1) Read training labels (CSV = ground truth, parent ids)
train_path = os.path.join(DATA_ROOT, 'train.csv')
train_df = pd.read_csv(train_path)
assert {'id','SMILES'}.issubset(train_df.columns), "train.csv must have id and SMILES"
train_df['id'] = train_df['id'].astype(int)

# 2) LMDB ids (these are augmented key_ids)
def read_lmdb_ids(lmdb_path: str) -> np.ndarray:
    ids_txt = lmdb_path + ".ids.txt"
    if not os.path.exists(ids_txt):
        raise FileNotFoundError(f"Missing {ids_txt}. Rebuild LMDB or confirm paths.")
    ids = np.loadtxt(ids_txt, dtype=np.int64)
    if ids.ndim == 0:
        ids = ids.reshape(1)
    return ids

lmdb_ids = read_lmdb_ids(TRAIN_LMDB)
print(f"LMDB contains {len(lmdb_ids):,} train graphs (augmented key_ids)")

# 3) parent map (better) or derive from key_ids
pmap_path = TRAIN_LMDB + ".parent_map.tsv"
if os.path.exists(pmap_path):
    pmap = pd.read_csv(pmap_path, sep="\t") # cols: key_id, parent_id, aug_idx, seed
    pmap['key_id'] = pmap['key_id'].astype(np.int64)
    pmap['parent_id'] = pmap['parent_id'].astype(np.int64)
else:
    # Fallback: derive parents from integer division
    pmap = pd.DataFrame({
        'key_id': lmdb_ids.astype(np.int64),
        'parent_id': (lmdb_ids // AUG_KEY_MULT).astype(np.int64),
    })

# 4) parents that actually exist in LMDB
parents_in_lmdb = np.sort(pmap['parent_id'].unique().astype(np.int64))

# 5) helper: parent ids that have a label for a given task
def parents_with_label(task: str) -> np.ndarray:
    col = task
    have_label = train_df.loc[~train_df[col].isna(), 'id'].astype(int).values
    keep = np.intersect1d(have_label, parents_in_lmdb, assume_unique=False)
    return keep

# 6) Make a parent-level global split once (reused for each task)
rng = np.random.default_rng(123)
perm = rng.permutation(len(parents_in_lmdb))
split = int(0.9 * len(parents_in_lmdb))
parents_train = parents_in_lmdb[perm[:split]]
parents_val = parents_in_lmdb[perm[split:]]

# Map parent split to augmented key_ids for the GNN
train_pool_key_ids = pmap.loc[pmap.parent_id.isin(parents_train), 'key_id'].astype(np.int64).values
val_pool_key_ids   = pmap.loc[pmap.parent_id.isin(parents_val), 'key_id'].astype(np.int64).values

print(f"Global pools -> train_pool={len(train_pool_key_ids):,}  val_pool={len(val_pool_key_ids):,}")
for t in label_cols:
    n = len(parents_with_label(t))
    print(f"{t:>7}: {n:6d} parents with labels (pre-intersection-by-task)")

In [ ]:
HOMO_CSV = os.path.join(DATA_ROOT, "homolumo_parent.csv")
homo = pd.read_csv(HOMO_CSV).drop_duplicates("parent_id").set_index("parent_id").sort_index()
embed_cols = [c for c in homo.columns if c.startswith("h_embed_")]

def append_homolumo_features(
        X_base: np.ndarray,
        parents_vec: np.ndarray,
        homo_df: pd.DataFrame,
        *,
        use_gap: bool = True,
        use_embed: bool = False,
        ) -> Tuple[np.ndarray, List[str]]:
    """
    Append HOMO-LUMO features to X_base using parent_ids in parents_vec.
    Toggles:
      - use_gap:    adds 1 col 'gap_pred'
      - use_embed:  adds all 'h_embed_*' cols
    """
    cols: List[str] = []
    blocks: List[np.ndarray] = []

    if use_gap:
        g = homo_df.reindex(parents_vec)["gap_pred"].to_numpy(dtype=np.float32).reshape(-1, 1)
        blocks.append(g)
        cols.append("gap_pred")

    if use_embed:
        embed_cols = [c for c in homo_df.columns if c.startswith("h_embed_")]
        if embed_cols:
            E = homo_df.reindex(parents_vec)[embed_cols].to_numpy(dtype=np.float32)
            blocks.append(E)
            cols.extend(embed_cols)

    if blocks:
        H = np.hstack(blocks)
        H = np.nan_to_num(H, nan=0.0, posinf=0.0, neginf=0.0)
        X_aug = np.hstack([X_base.astype(np.float32), H]).astype(np.float32)
        return X_aug, cols
    else:
        return X_base.astype(np.float32), cols

In [ ]:
def _safe_numpy(x, default_shape=None, dtype=np.float32):
    try:
        return torch.as_tensor(x).detach().cpu().numpy().astype(dtype)
    except Exception:
        if default_shape is None:
            return np.array([], dtype=dtype)
        return np.zeros(default_shape, dtype=dtype)

def geom_features_from_rec(rec, rdkit_dim_expected=15, rbf_K=32) -> np.ndarray:
    """
    Build a fixed-length vector from a single LMDB record:
      [rdkit(15), n_atoms, n_bonds, deg_mean, deg_max, has_xyz, eig3(3), bbox_extents(3), radius_stats(3), hop_hist(3), extra_atom_mean(5), edge_rbf_mean(32)]
    ~ total len = 15 + 5 + 3 + 3 + 3 + 3 + 5 + 32 = 69
    """
    # 15 RDKit descriptors stored in LMDB 
    rd = getattr(rec, "rdkit_feats", None)
    rd = _safe_numpy(rd, default_shape=(1, rdkit_dim_expected)).reshape(-1)
    if rd.size != rdkit_dim_expected:
        rd = np.zeros((rdkit_dim_expected,), dtype=np.float32)

    # basic graph sizes & degree
    x = torch.as_tensor(rec.x) # [N, ...]
    ei = torch.as_tensor(rec.edge_index) # [2, E]
    n = x.shape[0]
    e = ei.shape[1] if ei.ndim == 2 else 0
    deg = torch.bincount(ei[0], minlength=n) if e > 0 else torch.zeros(n, dtype=torch.long)
    deg_mean = deg.float().mean().item() if n > 0 else 0.0
    deg_max  = deg.max().item() if n > 0 else 0.0

    # has_xyz flag
    has_xyz = int(bool(getattr(rec, "has_xyz", torch.zeros(1, dtype=torch.bool))[0].item())) if hasattr(rec, "has_xyz") else 0

    # pos-based features
    eig3 = np.zeros(3, dtype=np.float32)
    extents = np.zeros(3, dtype=np.float32)
    rad_stats = np.zeros(3, dtype=np.float32)
    pos = getattr(rec, "pos", None)
    if pos is not None and n > 0 and has_xyz:
        P = torch.as_tensor(pos).float() # [N,3]
        center = P.mean(dim=0, keepdim=True)
        C = P - center
        cov = (C.T @ C) / max(1, n-1) # [3,3]
        vals = torch.linalg.eigvalsh(cov).clamp_min(0).sqrt() # length scales
        eig3 = vals.detach().cpu().numpy()
        mn, mx = P.min(0).values, P.max(0).values
        extents = (mx - mn).detach().cpu().numpy()
        r = C.norm(dim=1)
        rad_stats = np.array([r.mean().item(), r.std().item(), r.max().item()], dtype=np.float32)

    # hop-distance histogram (1,2,3 hops)
    hop_hist = np.zeros(3, dtype=np.float32)
    D = getattr(rec, "dist", None)
    if D is not None and n > 0:
        Dn = torch.as_tensor(D).float()[:n, :n]
        hop_hist = np.array([
            (Dn == 1).float().mean().item(),
            (Dn == 2).float().mean().item(),
            (Dn == 3).float().mean().item()
        ], dtype=np.float32)

    # extra atom features (mean over atoms, 5 dims if present)
    extra_atom = getattr(rec, "extra_atom_feats", None)
    extra_mean = np.zeros(5, dtype=np.float32)
    if extra_atom is not None and hasattr(extra_atom, "shape") and extra_atom.shape[-1] == 5:
        extra_mean = torch.as_tensor(extra_atom).float().mean(dim=0).detach().cpu().numpy()

    # edge RBF (last 32 channels of edge_attr were RBF(d))
    rbf_mean = np.zeros(rbf_K, dtype=np.float32)
    ea = getattr(rec, "edge_attr", None)
    if ea is not None:
        EA = torch.as_tensor(ea)
        if EA.ndim == 2 and EA.shape[1] >= (3 + rbf_K):
            rbf = EA[:, -rbf_K:].float()
            rbf_mean = rbf.mean(dim=0).detach().cpu().numpy()

    scalars = np.array([n, e, deg_mean, deg_max, has_xyz], dtype=np.float32)
    return np.concatenate([rd, scalars, eig3, extents, rad_stats, hop_hist, extra_mean, rbf_mean], axis=0)

In [ ]:
def morgan_bits(smiles_list, n_bits=1024, radius=3):
    X = np.zeros((len(smiles_list), n_bits), dtype=np.uint8)
    for i, s in enumerate(smiles_list):
        arr = np.zeros((n_bits,), dtype=np.uint8)
        m = Chem.MolFromSmiles(s)
        if m is not None:
            fp = rdmd.GetMorganFingerprintAsBitVect(m, radius=radius, nBits=n_bits)
            DataStructs.ConvertToNumpyArray(fp, arr)
        X[i] = arr
    return X.astype(np.float32)

def build_rf_features_from_lmdb(ids: np.ndarray, lmdb_path: str, smiles_list: List[str]) -> np.ndarray:
    """
    Returns X = [Morgan1024 | LMDB-3D-global(69)] for each id/smiles.
    Assumes ids and smiles_list are aligned with the CSV used to build LMDB.
    """
    base = LMDBDataset(ids, lmdb_path)
    # 3D/global block
    feats3d = []
    for i in range(len(base)):
        rec = base[i]
        feats3d.append(geom_features_from_rec(rec)) # shape (69,)
    X3d = np.vstack(feats3d).astype(np.float32) if feats3d else np.zeros((0, 69), dtype=np.float32)

    # Morgan FP block (2D)
    Xfp = morgan_bits(smiles_list, n_bits=1024, radius=3)# (N,1024)

    # concat
    X = np.hstack([Xfp, X3d]).astype(np.float32)# (N, 1024+69)
    return X

# Models

In [ ]:
# Build parent-level target DataFrames for RF/tabular
train_df = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))
train_df["id"] = train_df["id"].astype(int)

# Reuse parents_in_lmdb computed above
def build_target_df_from_parents(df: pd.DataFrame, target_col: str, keep_parent_ids: np.ndarray):
    """
    Returns DataFrame with ['id','SMILES', target_col] restricted to PARENT ids that
    exist in the LMDB; drops missing targets.
    """
    out = df.loc[df["id"].isin(keep_parent_ids), ["id", "SMILES", target_col]].copy()
    print(f"Initial {target_col} shape:", out.shape)
    print(f"Initial {target_col} missing:\n{out.isnull().sum()}")
    out = out.dropna(subset=[target_col]).reset_index(drop=True)
    print(f"Cleaned {target_col} shape:", out.shape)
    print(f"Cleaned {target_col} missing:\n{out.isnull().sum()}\n")
    return out

# Build all five on PARENTS that exist in LMDB
df_tg = build_target_df_from_parents(train_df, "Tg", parents_in_lmdb)
df_density = build_target_df_from_parents(train_df, "Density", parents_in_lmdb)
df_ffv = build_target_df_from_parents(train_df, "FFV", parents_in_lmdb)
df_tc = build_target_df_from_parents(train_df, "Tc", parents_in_lmdb)
df_rg = build_target_df_from_parents(train_df, "Rg", parents_in_lmdb)

In [ ]:
# Morgan FP utilities (no 3D, no external descriptors) 
def smiles_to_morgan_fp(
        smi: str,
        n_bits: int = 1024,
        radius: int = 3,
        use_counts: bool = False,
        ) -> Optional[np.ndarray]:
    """Return a 1D numpy array Morgan fingerprint; None if SMILES invalid."""
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    if use_counts:
        fp = rdmd.GetMorganFingerprint(mol, radius)
        # convert to dense count vector
        arr = np.zeros((n_bits,), dtype=np.int32)
        for bit_id, count in fp.GetNonzeroElements().items():
            arr[bit_id % n_bits] += count
        return arr.astype(np.float32)
    else:
        bv = rdmd.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=np.int8)
        Chem.DataStructs.ConvertToNumpyArray(bv, arr)
        return arr.astype(np.float32)

def prepare_fp_for_target(
        df_target: pd.DataFrame,
        target_col: str,
        *,
        fp_bits: int = 1024,
        fp_radius: int = 3,
        use_counts: bool = False,
        save_csv_path: Optional[str] = None,
        show_progress: bool = True,
        ) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    """
    Drop missing targets, compute Morgan FPs from SMILES only.
    Returns (df_clean, y, X_fp) where:
      df_clean: ['SMILES', target_col]
      y: (N,)
      X_fp: (N, fp_bits)
    """
    assert {"SMILES", target_col}.issubset(df_target.columns)

    # 1) drop missing targets 
    work = df_target[["SMILES", target_col]].copy()
    before = len(work)
    work = work.dropna(subset=[target_col]).reset_index(drop=True)
    after = len(work)
    print(f"[{target_col}] dropped {before - after} missing; kept {after}")

    # 2) compute FPs and skip invalid SMILES
    fps, ys, keep_smiles = [], [], []
    it = work.itertuples(index=False)
    if show_progress:
        it = tqdm(it, total=len(work), desc=f"FPs for {target_col}")

    for row in it:
        smi = row.SMILES
        yv  = getattr(row, target_col)
        arr = smiles_to_morgan_fp(smi, n_bits=fp_bits, radius=fp_radius, use_counts=use_counts)
        if arr is None:
            continue
        fps.append(arr)
        ys.append(float(yv))
        keep_smiles.append(smi)

    X_fp = np.stack(fps, axis=0) if fps else np.zeros((0, fp_bits), dtype=np.float32)
    y = np.asarray(ys, dtype=float)
    df_clean = pd.DataFrame({"SMILES": keep_smiles, target_col: y})

    if save_csv_path:
        df_clean.to_csv(save_csv_path, index=False)
        print(f"[{target_col}] saved cleaned CSV -> {save_csv_path}")

    print(f"[{target_col}] X_fp: {X_fp.shape} | y: {y.shape}")
    return df_clean, y, X_fp

In [ ]:
# Bit vectors (1024, r=3) 
df_clean_tg, y_tg, X_tg = prepare_fp_for_target(df_tg, "Tg", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tg_fp.csv")
df_clean_density, y_density, X_density = prepare_fp_for_target(df_density, "Density", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_density_fp.csv")
df_clean_ffv, y_ffv, X_ffv = prepare_fp_for_target(df_ffv, "FFV", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_ffv_fp.csv")
df_clean_tc, y_tc, X_tc = prepare_fp_for_target(df_tc, "Tc", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tc_fp.csv")
df_clean_rg, y_rg, X_rg = prepare_fp_for_target(df_rg, "Rg", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_rg_fp.csv")

In [ ]:
from dataclasses import dataclass
@dataclass
class TabularSplits:
    # unscaled (for RF)
    X_train: np.ndarray
    X_test:  np.ndarray
    y_train: np.ndarray
    y_test:  np.ndarray
    # scaled (for KRR/MLP)
    X_train_scaled: Optional[np.ndarray] = None
    X_test_scaled:  Optional[np.ndarray] = None
    y_train_scaled: Optional[np.ndarray] = None  # shape (N,1)
    y_test_scaled:  Optional[np.ndarray] = None
    x_scaler: Optional[StandardScaler] = None
    y_scaler: Optional[StandardScaler] = None

def _make_regression_stratify_bins(y: np.ndarray, n_bins: int = 10) -> np.ndarray:
    """Return integer bins for approximate stratification in regression."""
    y = y.ravel()
    # handle degenerate case
    if np.unique(y).size < n_bins:
        n_bins = max(2, np.unique(y).size)
    quantiles = np.linspace(0, 1, n_bins + 1)
    bins = np.unique(np.quantile(y, quantiles))
    # ensure strictly increasing
    bins = np.unique(bins)
    # np.digitize expects right-open intervals by default
    strat = np.digitize(y, bins[1:-1], right=False)
    return strat

def make_tabular_splits(
        X: np.ndarray,
        y: np.ndarray,
        *,
        test_size: float = 0.2,
        random_state: int = 42,
        scale_X: bool = True,
        scale_y: bool = True,
        stratify_regression: bool = False,
        n_strat_bins: int = 10,
        train_idx: Optional[np.ndarray] = None,
        test_idx: Optional[np.ndarray] = None,
        ) -> TabularSplits:
    """
    Split and (optionally) scale tabular features/targets for a single target.
    Returns both scaled and unscaled arrays, plus fitted scalers.
    """
    y = np.asarray(y, dtype=float).ravel()
    X = np.asarray(X)

    if train_idx is not None and test_idx is not None:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    else:
        strat = None
        if stratify_regression:
            strat = _make_regression_stratify_bins(y, n_bins=n_strat_bins)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=strat
        )

    # Unscaled outputs (for RF, tree models)
    splits = TabularSplits(
        X_train=X_train, X_test=X_test,
        y_train=y_train, y_test=y_test
    )

    # Scaled versions (for KRR/MLP)
    if scale_X:
        xscaler = StandardScaler()
        splits.X_train_scaled = xscaler.fit_transform(X_train)
        splits.X_test_scaled  = xscaler.transform(X_test)
        splits.x_scaler = xscaler
    if scale_y:
        yscaler = StandardScaler()
        splits.y_train_scaled = yscaler.fit_transform(y_train.reshape(-1, 1))
        splits.y_test_scaled  = yscaler.transform(y_test.reshape(-1, 1))
        splits.y_scaler = yscaler

    print("Splits:")
    print("X_train:", splits.X_train.shape, "| X_test:", splits.X_test.shape)
    if splits.X_train_scaled is not None:
        print("X_train_scaled:", splits.X_train_scaled.shape, "| X_test_scaled:", splits.X_test_scaled.shape)
    print("y_train:", splits.y_train.shape, "| y_test:", splits.y_test.shape)
    if splits.y_train_scaled is not None:
        print("y_train_scaled:", splits.y_train_scaled.shape, "| y_test_scaled:", splits.y_test_scaled.shape)

    return splits

In [ ]:
def _reg_metrics(y_tr, p_tr, y_va, p_va):
    return {
        "train_MAE": mean_absolute_error(y_tr, p_tr),
        "train_RMSE": mean_squared_error(y_tr, p_tr),
        "train_R2": r2_score(y_tr, p_tr),
        "val_MAE": mean_absolute_error(y_va, p_va),
        "val_RMSE": mean_squared_error(y_va, p_va),
        "val_R2": r2_score(y_va, p_va),
    }

In [ ]:
def train_eval_rf(
        X: np.ndarray,
        y: np.ndarray,
        *,
        rf_params: Dict[str, Any],
        test_size: float = 0.2,
        random_state: int = 42,
        stratify_regression: bool = True,
        n_strat_bins: int = 10,
        save_dir: str = os.path.join(PROJECT_ROOT, "saved_models", "baseline", "RandomForest", "rf"),
        tag: str = "model",
        ) -> Tuple[RandomForestRegressor, Dict[str, float], TabularSplits, str]:
    """
    Trains a RandomForest on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)
    if stratify_regression:
        adaptive_bins = min(n_strat_bins, max(3, int(np.sqrt(len(y)))))
    else:
        adaptive_bins = n_strat_bins
    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False, # RF doesn't need scaling
        stratify_regression=stratify_regression,
        n_strat_bins=adaptive_bins
    )

    rf = RandomForestRegressor(random_state=random_state, n_jobs=-1, **rf_params)
    rf.fit(splits.X_train, splits.y_train)

    pred_tr = rf.predict(splits.X_train)
    pred_te = rf.predict(splits.X_test)
    metrics = _reg_metrics(splits.y_train, pred_tr, splits.y_test, pred_te)
    print(f"[RF/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"rf_{tag}.joblib")
    joblib.dump({"model": rf, "metrics": metrics, "rf_params": rf_params}, path)
    return rf, metrics, splits, path

def train_eval_lgbm(
        X, y, *,
        lgbm_params,
        test_size=0.2, 
        random_state=42,
        stratify_regression=True, 
        n_strat_bins=10,
        save_dir=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "LightGBM", "lgbm"),
        tag="model",
        early_stopping_rounds=400,
        extra_dump: Optional[Dict[str,Any]]=None
        ):
    os.makedirs(save_dir, exist_ok=True)
    adaptive_bins = min(n_strat_bins, max(3, int(np.sqrt(len(y))))) if stratify_regression else n_strat_bins
    splits = make_tabular_splits(
        X, y, test_size=test_size, random_state=random_state,
        scale_X=False, scale_y=False,
        stratify_regression=stratify_regression, n_strat_bins=adaptive_bins
    )

    Xtr = np.asarray(splits.X_train, dtype=np.float32)
    Ytr = np.asarray(splits.y_train, dtype=np.float32)
    Xva = np.asarray(splits.X_test, dtype=np.float32)
    Yva = np.asarray(splits.y_test, dtype=np.float32)

    base = dict(
        n_estimators=4000,
        learning_rate=0.03,
        objective="l1",            
        random_state=random_state,
        n_jobs=-1,
        verbosity=-1,             
    )
    lgb_params = {k: v for k, v in lgbm_params.items() if k not in ("colsample_bytree", "subsample", "subsample_freq")}
    # if no bagging, drop bagging_freq to avoid warning
    if lgb_params.get("bagging_fraction", 1.0) >= 1.0:
        lgb_params.pop("bagging_freq", None)
    base.update(lgb_params)

    lgbm = lgb.LGBMRegressor(**base)
    lgbm.fit(
        Xtr, Ytr,
        eval_set=[(Xva, Yva)],
        eval_metric="l1",
        callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=False),
                   lgb.log_evaluation(period=0)]
    )

    p_tr = lgbm.predict(Xtr, num_iteration=lgbm.best_iteration_)
    p_va = lgbm.predict(Xva, num_iteration=lgbm.best_iteration_)
    metrics = _reg_metrics(Ytr, p_tr, Yva, p_va)
    print(f"[LGBM/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"lgbm_{tag}.joblib")
    payload = {"model": lgbm, "metrics": metrics, "lgbm_params": base}
    if extra_dump: payload.update(extra_dump)       # <<<
    joblib.dump(payload, path)
    return lgbm, metrics, splits, path


def train_eval_xgb(
    X: np.ndarray,
    y: np.ndarray,
    *,
    xgb_params: Dict[str, Any],
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    save_dir: str=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "XGBoost", "xgb"),
    tag: str = "model",
) -> Tuple[xgb.XGBRegressor, Dict[str, float], TabularSplits, str]:
    """
    Trains an XGBoost on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)
    if stratify_regression:
        adaptive_bins = min(n_strat_bins, max(3, int(np.sqrt(len(y)))))
    else:
        adaptive_bins = n_strat_bins
    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,
        stratify_regression=stratify_regression,
        n_strat_bins=adaptive_bins
    )
    
    # Base parameters as provided by the user
    base_params = dict(
        n_estimators=6000,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.8,
        colsample_bylevel=0.8,
        colsample_bynode=0.8,
        reg_lambda=2.0,
        reg_alpha=0.0,
        min_child_weight=2.0,
        gamma=0.0,
        # Check for GPU availability and set tree_method accordingly
        # This requires the 'xgboost' library to be installed with GPU support
        tree_method="gpu_hist" if "cuda" in xgb_params.get("device", "cpu") else "hist",
        max_bin=512,
        objective="reg:squarederror",
        eval_metric="mae",
        random_state=random_state,
    )
    
    # Merge base and custom parameters, with custom ones taking precedence
    full_params = {**base_params, **xgb_params}

    xgbr = xgb.XGBRegressor(**full_params)
    
    # Removed early stopping to avoid `TypeError` due to library version incompatibility.
    xgbr.fit(
        splits.X_train,
        splits.y_train,
        eval_set=[(splits.X_test, splits.y_test)],
    )

    pred_tr = xgbr.predict(splits.X_train)
    pred_te = xgbr.predict(splits.X_test)
    metrics = _reg_metrics(splits.y_train, pred_tr, splits.y_test, pred_te)
    print(f"[XGB/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"xgb_{tag}.joblib")
    joblib.dump({"model": xgbr, "metrics": metrics, "xgb_params": xgb_params}, path)
    return xgbr, metrics, splits, path

In [ ]:
rf_cfg = {
"FFV": {"n_estimators": 100, "max_depth": 60},
"Tc": {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
"Rg": {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
}

rf_cfg["Tg"] = rf_cfg["Rg"]
rf_cfg["Density"] = rf_cfg["Rg"]

lgbm_cfg = {
"FFV": {"num_leaves": 127, "min_child_samples": 20, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
"Tc": {"num_leaves": 63,  "min_child_samples": 15, "feature_fraction": 0.7, "bagging_fraction": 0.8, "bagging_freq": 1},
"Rg": {"num_leaves": 127, "min_child_samples": 10, "feature_fraction": 1.0, "bagging_fraction": 0.8, "bagging_freq": 1},
"Tg": {"num_leaves": 127, "min_child_samples": 20, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
"Density": {"num_leaves": 63,  "min_child_samples": 10, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
}

xgb_cfg = {
"FFV": {"max_depth": 6,  "min_child_weight": 2.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.85},
"Tc": {"max_depth": 6,  "min_child_weight": 4.0, "gamma": 0.1, "reg_lambda": 3.0, "reg_alpha": 0.1, "colsample_bytree": 0.8},
"Rg": {"max_depth": 10, "min_child_weight": 2.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.9},
"Tg": {"max_depth": 10, "min_child_weight": 4.0, "gamma": 0.2, "reg_lambda": 3.0, "reg_alpha": 0.1, "colsample_bytree": 0.85},
"Density": {"max_depth": 6,  "min_child_weight": 1.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.8},
}


print("\n--- Training Random Forest Models ---")
rf_ffv, m_ffv, splits_ffv, p_ffv = train_eval_rf(X_ffv, y_ffv, rf_params=rf_cfg["FFV"], tag="FFV")
rf_tc, m_tc, splits_tc, p_tc = train_eval_rf(X_tc, y_tc, rf_params=rf_cfg["Tc"], tag="Tc")
rf_rg, m_rg, splits_rg, p_rg = train_eval_rf(X_rg, y_rg, rf_params=rf_cfg["Rg"], tag="Rg")
rf_tg, m_tg, splits_tg, p_tg = train_eval_rf(X_tg, y_tg, rf_params=rf_cfg["Tg"], tag="Tg")
rf_density, m_density, splits_density, p_density = train_eval_rf(X_density, y_density, rf_params=rf_cfg["Density"], tag="Density")

print("\n--- Training LightGBM Models ---")
lgbm_ffv, m_lgbm_ffv, splits_lgbm_ffv, p_lgbm_ffv = train_eval_lgbm(X_ffv, y_ffv, lgbm_params=lgbm_cfg["FFV"], tag="FFV")
lgbm_tc, m_lgbm_tc, splits_lgbm_tc, p_lgbm_tc = train_eval_lgbm(X_tc, y_tc, lgbm_params=lgbm_cfg["Tc"], tag="Tc")
lgbm_rg, m_lgbm_rg, splits_lgbm_rg, p_lgbm_rg = train_eval_lgbm(X_rg, y_rg, lgbm_params=lgbm_cfg["Rg"], tag="Rg")
lgbm_tg, m_lgbm_tg, splits_lgbm_tg, p_lgbm_tg = train_eval_lgbm(X_tg, y_tg, lgbm_params=lgbm_cfg["Tg"], tag="Tg")
lgbm_density, m_lgbm_density, splits_lgbm_density, p_lgbm_density = train_eval_lgbm(X_density, y_density, lgbm_params=lgbm_cfg["Density"], tag="Density")

print("\n--- Training XGBoost Models ---")
xgb_ffv, m_xgb_ffv, splits_xgb_ffv, p_xgb_ffv = train_eval_xgb(X_ffv, y_ffv, xgb_params=xgb_cfg["FFV"], tag="FFV")
xgb_tc, m_xgb_tc, splits_xgb_tc, p_xgb_tc = train_eval_xgb(X_tc, y_tc, xgb_params=xgb_cfg["Tc"], tag="Tc")
xgb_rg, m_xgb_rg, splits_xgb_rg, p_xgb_rg = train_eval_xgb(X_rg, y_rg, xgb_params=xgb_cfg["Rg"], tag="Rg")
xgb_tg, m_xgb_tg, splits_xgb_tg, p_xgb_tg = train_eval_xgb(X_tg, y_tg, xgb_params=xgb_cfg["Tg"], tag="Tg")
xgb_density, m_xgb_density, splits_xgb_density, p_xgb_density = train_eval_xgb(X_density, y_density, xgb_params=xgb_cfg["Density"], tag="Density")

In [ ]:
def build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_list, *, agg="mean"):
    """
    Uses your EXISTING build_rf_features_from_lmdb(ids, lmdb_path, smiles_list)
    but expands each parent id -> its augmented key_ids, extracts features for all,
    then reduces (mean/median) back to ONE row per parent (so shapes match y).

    Returns:
        X_parent  : (N_parents_kept, D)   aggregated features per parent
        keep_idx  : indices into the input arrays (parent_ids/smiles_list/y)
                    that were actually kept (in order)
    """
    pmap_path = lmdb_path + ".parent_map.tsv"
    if os.path.exists(pmap_path):
        pmap = pd.read_csv(pmap_path, sep="\t")  # cols: key_id, parent_id, aug_idx, seed
        pmap['key_id'] = pmap['key_id'].astype(np.int64)
        pmap['parent_id'] = pmap['parent_id'].astype(np.int64)
        group = pmap.groupby('parent_id')['key_id'].apply(list).to_dict()
    else:
        # derive from ids.txt if parent_map.tsv is missing
        lmdb_ids = np.loadtxt(lmdb_path + ".ids.txt", dtype=np.int64)
        if lmdb_ids.ndim == 0: lmdb_ids = lmdb_ids.reshape(1)
        dfmap = pd.DataFrame({
            'parent_id': (lmdb_ids // AUG_KEY_MULT).astype(np.int64),
            'key_id': lmdb_ids.astype(np.int64),
        })
        group = dfmap.groupby('parent_id')['key_id'].apply(list).to_dict()

    # 2) expand to augmented keys while tracking slices
    flat_keys, flat_smiles, seg_sizes = [], [], []
    for pid, smi in zip(parent_ids, smiles_list):
        keys = group.get(int(pid), [])
        seg_sizes.append(len(keys))
        if len(keys):
            flat_keys.extend(keys)
            flat_smiles.extend([smi] * len(keys))

    if len(flat_keys) == 0:
        raise ValueError("No augmented key_ids found for provided parent ids. "
                         "Check that LMDB matches this CSV and AUG_KEY_MULT.")

    X_all = build_rf_features_from_lmdb(np.array(flat_keys, dtype=np.int64), lmdb_path, flat_smiles)
    # 4) fold back to parents by aggregation
    out_rows, keep_idx = [], []
    i0 = 0
    for i, k in enumerate(seg_sizes):
        if k == 0: # parent not present in LMDB (should be rare)
            continue
        Xi = X_all[i0:i0+k]
        i0 += k
        if agg == "mean":
            out_rows.append(Xi.mean(axis=0))
        elif agg == "median":
            out_rows.append(np.median(Xi, axis=0))
        elif agg == "max":
            out_rows.append(Xi.max(axis=0))
        else:
            raise ValueError(f"Unsupported agg={agg}")
        keep_idx.append(i)

    X_parent = np.vstack(out_rows).astype(np.float32)
    keep_idx = np.asarray(keep_idx, dtype=int)

    assert X_parent.ndim == 2 and X_parent.shape[0] == keep_idx.size, "bad aggregation folding"

    n_drop = (len(parent_ids) - keep_idx.size)
    if n_drop:
        print(f"[build_rf_features_from_lmdb_parents] Dropped {n_drop} parents with 0 aug rows in LMDB")

    return X_parent, keep_idx

In [ ]:
def train_rf_aug3d_for_target(
        target_col: str,
        rf_params: dict,
        *,
        train_csv_path: str,
        lmdb_path: str,
        save_dir: str = os.path.join(PROJECT_ROOT, "saved_models", "baseline", "RandomForest", "rf_aug3d"),
        tag_prefix: str = "aug3D",
        test_size: float = 0.2,
        random_state: int = 42,
        stratify_regression: bool = True,
        n_strat_bins: int = 10,
        agg: str = "mean",
        homo_df: Optional[pd.DataFrame] = None,
        use_gap: bool = False,
        use_embed: bool = False,
        ):
    df = pd.read_csv(train_csv_path)
    mask = ~df[target_col].isna()
    parent_ids = df.loc[mask, 'id'].astype(int).values
    smiles_tr = df.loc[mask, 'SMILES'].astype(str).tolist()
    y = df.loc[mask, target_col].astype(float).values

    # LMDB features aggregated over augmentations
    X_parent, keep_idx = build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_tr, agg=agg)
    y_keep = y[keep_idx]

    # append HOMO–LUMO (toggle gap/embed)
    parents_kept = parent_ids[keep_idx]
    if homo_df is not None and (use_gap or use_embed):
        X_parent, added_cols = append_homolumo_features(
            X_parent, parents_kept, homo_df, use_gap=use_gap, use_embed=use_embed
        )
        print(f"[{target_col}] appended HOMO cols: {len(added_cols)} -> X={X_parent.shape}")

    model, metrics, splits, path = train_eval_rf(
        X_parent, y_keep,
        rf_params=rf_params,
        test_size=test_size,
        random_state=random_state,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins,
        save_dir=save_dir,
        tag=f"{target_col}_{tag_prefix}_{agg}" + ("+gap" if use_gap else "") + ("+emb" if use_embed else "")
        )
    return model, metrics, splits, path


def train_lgbm_aug3d_for_target(
        target_col: str,
        lgbm_params: dict,
        *,
        train_csv_path: str,
        lmdb_path: str,
        save_dir: str=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "LightGBM", "lgbm_aug3d"),
        tag_prefix: str = "aug3D",
        test_size: float = 0.2,
        random_state: int = 42,
        stratify_regression: bool = True,
        n_strat_bins: int = 10,
        agg: str = "mean",
        homo_df: Optional[pd.DataFrame] = None,
        use_gap: bool = False,
        use_embed: bool = False,
        ):
    """Load rows with target, build X=[FP|3D], train LGBM."""

    df = pd.read_csv(train_csv_path)
    mask = ~df[target_col].isna()
    parent_ids = df.loc[mask, 'id'].astype(int).values
    smiles_tr = df.loc[mask, 'SMILES'].astype(str).tolist()
    y = df.loc[mask, target_col].astype(float).values

    # LMDB features aggregated over augmentations
    X_parent, keep_idx = build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_tr, agg=agg)
    y_keep = y[keep_idx]

    # append HOMO–LUMO (toggle gap/embed)
    parents_kept = parent_ids[keep_idx]
    if homo_df is not None and (use_gap or use_embed):
        X_parent, added_cols = append_homolumo_features(
            X_parent, parents_kept, homo_df, use_gap=use_gap, use_embed=use_embed
        )
        print(f"[{target_col}] appended HOMO cols: {len(added_cols)} -> X={X_parent.shape}")


    model, metrics, splits, path = train_eval_lgbm(
        X_parent, y_keep,
        lgbm_params=lgbm_params,
        test_size=test_size,
        random_state=random_state,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins,
        save_dir=save_dir,
        tag=f"{target_col}_{tag_prefix}_{agg}" + ("+gap" if use_gap else "") + ("+emb" if use_embed else "")
    )
    return model, metrics, splits, path

def train_xgb_aug3d_for_target(
        target_col: str,
        xgb_params: dict,
        *,
        train_csv_path: str,
        lmdb_path: str,
        save_dir: str=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "XGBoost", "xgb_aug3d"),
        tag_prefix: str = "aug3D",
        test_size: float = 0.2,
        random_state: int = 42,
        stratify_regression: bool = True,
        n_strat_bins: int = 10,
        agg: str = "mean",
        homo_df: Optional[pd.DataFrame] = None,
        use_gap: bool = False,
        use_embed: bool = False,
        ):
    """Load rows with target, build X=[FP|3D], train XGB."""

    df = pd.read_csv(train_csv_path)
    mask = ~df[target_col].isna()
    parent_ids = df.loc[mask, 'id'].astype(int).values
    smiles_tr = df.loc[mask, 'SMILES'].astype(str).tolist()
    y = df.loc[mask, target_col].astype(float).values

    # LMDB features aggregated over augmentations
    X_parent, keep_idx = build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_tr, agg=agg)
    y_keep = y[keep_idx]

    # append HOMO–LUMO (toggle gap/embed)
    parents_kept = parent_ids[keep_idx]
    if homo_df is not None and (use_gap or use_embed):
        X_parent, added_cols = append_homolumo_features(
            X_parent, parents_kept, homo_df, use_gap=use_gap, use_embed=use_embed
        )
        print(f"[{target_col}] appended HOMO cols: {len(added_cols)} -> X={X_parent.shape}")

    model, metrics, splits, path = train_eval_xgb(
        X_parent, y_keep,
        xgb_params=xgb_params,
        test_size=test_size,
        random_state=random_state,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins,
        save_dir=save_dir,
        tag=f"{target_col}_{tag_prefix}_{agg}" + ("+gap" if use_gap else "") + ("+emb" if use_embed else "")
    )
    return model, metrics, splits, path

rf_cfg_aug = {
    "FFV": {"n_estimators": 800, "max_depth": 30, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Tc": {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg": {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
    "Tg": {"n_estimators": 600, "max_depth": 60, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Density": {"n_estimators": 600, "max_depth": 40, "min_samples_leaf": 1, "max_features": "sqrt"},
}

lgbm_cfg_aug = {
  "FFV": {"num_leaves": 127, "min_child_samples": 20, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
  "Tc": {"num_leaves": 63,  "min_child_samples": 15, "feature_fraction": 0.7, "bagging_fraction": 0.8, "bagging_freq": 1},
  "Rg": {"num_leaves": 127, "min_child_samples": 10, "feature_fraction": 1.0, "bagging_fraction": 0.8, "bagging_freq": 1},
  "Tg": {"num_leaves": 127, "min_child_samples": 20, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
  "Density": {"num_leaves": 63,  "min_child_samples": 10, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 1},
}
xgb_cfg_aug = {
  "FFV": {"max_depth": 6,  "min_child_weight": 2.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.85},
  "Tc": {"max_depth": 6,  "min_child_weight": 4.0, "gamma": 0.1, "reg_lambda": 3.0, "reg_alpha": 0.1, "colsample_bytree": 0.8},
  "Rg": {"max_depth": 10, "min_child_weight": 2.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.9},
  "Tg": {"max_depth": 10, "min_child_weight": 4.0, "gamma": 0.2, "reg_lambda": 3.0, "reg_alpha": 0.1, "colsample_bytree": 0.85},
  "Density": {"max_depth": 6,  "min_child_weight": 1.0, "gamma": 0.0, "reg_lambda": 2.0, "reg_alpha": 0.0, "colsample_bytree": 0.8},
}


TRAIN_CSV = os.path.join(DATA_ROOT, "train.csv")
targets = ["FFV", "Tc", "Rg", "Tg", "Density"]
print("--- Training Random Forest (+3D) Models ---")
rf_models, rf_metrics, rf_splits, rf_paths = {}, {}, {}, {}
for t in targets:
    print(f"\n>>> Training RF(+3D) for {t}")
    m, met, sp, p = train_rf_aug3d_for_target(
        t, rf_cfg_aug[t],
        train_csv_path=TRAIN_CSV,
        lmdb_path=TRAIN_LMDB,
        save_dir=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "RandomForest", "rf_aug3d"),
        tag_prefix="aug3D"
    )
    rf_models[t], rf_metrics[t], rf_splits[t], rf_paths[t] = m, met, sp, p
    print(f"[RF+3D/{t}]  val_MAE={met['val_MAE']:.6f}  val_RMSE={met['val_RMSE']:.6f}  val_R2={met['val_R2']:.4f}")

print("\n--- Training LightGBM (+3D) Models ---")
lgbm_models, lgbm_metrics, lgbm_splits, lgbm_paths = {}, {}, {}, {}
for t in targets:
    print(f"\n>>> Training LGBM(+3D) for {t}")
    m, met, sp, p = train_lgbm_aug3d_for_target(
        t, lgbm_cfg_aug[t],
        train_csv_path=TRAIN_CSV,
        lmdb_path=TRAIN_LMDB,
        save_dir=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "LightGBM", "lgbm_aug3d"),
        tag_prefix="aug3D"
    )
    lgbm_models[t], lgbm_metrics[t], lgbm_splits[t], lgbm_paths[t] = m, met, sp, p
    print(f"[LGBM+3D/{t}]  val_MAE={met['val_MAE']:.6f}  val_RMSE={met['val_RMSE']:.6f}  val_R2={met['val_R2']:.4f}")

print("\n--- Training XGBoost (+3D) Models ---")
xgb_models, xgb_metrics, xgb_splits, xgb_paths = {}, {}, {}, {}
for t in targets:
    print(f"\n>>> Training XGB(+3D) for {t}")
    m, met, sp, p = train_xgb_aug3d_for_target(
        t, xgb_cfg_aug[t],
        train_csv_path=TRAIN_CSV,
        lmdb_path=TRAIN_LMDB,
        save_dir=os.path.join(PROJECT_ROOT, "saved_models", "baseline", "XGBoost", "xgb_aug3d"),
        tag_prefix="aug3D"
    )
    xgb_models[t], xgb_metrics[t], xgb_splits[t], xgb_paths[t] = m, met, sp, p
    print(f"[XGB+3D/{t}]  val_MAE={met['val_MAE']:.6f}  val_RMSE={met['val_RMSE']:.6f}  val_R2={met['val_R2']:.4f}")

# GNN

In [ ]:
# Fixed version - use the parent mapping approach
label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx   = {k:i for i,k in enumerate(label_cols)}
AUG_KEY_MULT = 1000  

train_csv = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))
train_csv["id"] = train_csv["id"].astype(int)

# Load parent mapping (key_id -> parent_id)
pmap_path = TRAIN_LMDB + ".parent_map.tsv"
if os.path.exists(pmap_path):
    pmap = pd.read_csv(pmap_path, sep="\t") # cols: key_id, parent_id, aug_idx, seed
    pmap["key_id"] = pmap["key_id"].astype(np.int64)
    pmap["parent_id"] = pmap["parent_id"].astype(np.int64)
    parents_in_lmdb = np.sort(pmap["parent_id"].unique().astype(np.int64))
else:
    # fallback: derive parent IDs from synthetic IDs
    lmdb_ids_path = TRAIN_LMDB + ".ids.txt"
    lmdb_ids = np.loadtxt(lmdb_ids_path, dtype=np.int64)
    if lmdb_ids.ndim == 0: lmdb_ids = lmdb_ids.reshape(1)
    parents_in_lmdb = np.sort((lmdb_ids // AUG_KEY_MULT).unique().astype(np.int64))

def ids_for_task(task):
    t = task2idx[task]
    col = label_cols[t]
    # Get parent IDs that have the target and exist in LMDB
    parent_ids = train_csv.loc[~train_csv[col].isna(), 'id'].astype(int).tolist()
    # Filter to only those that exist in LMDB
    valid_parent_ids = [pid for pid in parent_ids if pid in parents_in_lmdb]
    
    # Convert to synthetic IDs (use first conformer: aug_idx=0)
    synthetic_ids = [pid * AUG_KEY_MULT + 0 for pid in valid_parent_ids]
    return np.array(synthetic_ids, dtype=np.int64)

ids_tg = ids_for_task("Tg")
ids_den = ids_for_task("Density")
ids_tc = ids_for_task("Tc")
ids_rg = ids_for_task("Rg")
ids_ffv = ids_for_task("FFV")
print("Tg ids:", ids_tg.shape, "Density ids:", ids_den.shape)

In [ ]:
def _get_rdkit_feats_from_record(rec):
    arr = getattr(rec, "rdkit_feats", None)
    if arr is None:
        return torch.zeros(15, dtype=torch.float32) # or 6 if depending on LMDB build
    v = torch.as_tensor(np.asarray(arr, np.float32).reshape(-1), dtype=torch.float32)
    return v.unsqueeze(0) # (1, D) so batch -> (B, D)


class LMDBtoPyGSingleTask(Dataset):
    def __init__(
            self,
            ids,
            lmdb_path,
            target_index=None,
            *,
            use_mixed_edges: bool = True, # <— 3 cat + 32 RBF continuous
            include_extra_atom_feats: bool = True, 
            ):
        self.base = LMDBDataset(ids, lmdb_path)
        self.t = target_index
        self.use_mixed_edges = use_mixed_edges
        self.include_extra_atom_feats = include_extra_atom_feats

    def __len__(self): 
        return len(self.base)

    def __getitem__(self, idx):
        rec = self.base[idx]

        x = torch.as_tensor(rec.x, dtype=torch.long)
        ei = torch.as_tensor(rec.edge_index, dtype=torch.long)

        ea = torch.as_tensor(rec.edge_attr) # (E, 3 + 32)
        if self.use_mixed_edges:
            # keep all columns, EdgeEncoderMixed will split cat vs cont
            edge_attr = ea.to(torch.float32)
        else:
            # categorical-only for BondEncoder
            edge_attr = ea[:, :3].to(torch.long)

        # rdkit globals: Keep as (1, D) so PyG collates to (B, D)
        rdkit_feats = _get_rdkit_feats_from_record(rec) # (1, D)
        d = Data(x=x, edge_index=ei, edge_attr=edge_attr, rdkit_feats=rdkit_feats)

        if self.include_extra_atom_feats and hasattr(rec, "extra_atom_feats"):
            d.extra_atom_feats = torch.as_tensor(rec.extra_atom_feats, dtype=torch.float32) # (N,5)

        if hasattr(rec, "has_xyz"):
            # collates to (B,1), handy as a gating/global indicator
            hz = np.asarray(rec.has_xyz, np.uint8).reshape(-1)
            d.has_xyz = torch.from_numpy(hz.astype(np.float32))

        if (self.t is not None) and hasattr(rec, "y"):
            yv = torch.as_tensor(rec.y, dtype=torch.float32).view(-1)
            if self.t < yv.numel():
                d.y = yv[self.t:self.t+1] # (1,)

        # geometry & extras from LMDB (if present)
        if hasattr(rec, "pos"): # (N,3) float
            d.pos = torch.as_tensor(rec.pos, dtype=torch.float32)
        if hasattr(rec, "extra_atom_feats"):# (N,5) float
            d.extra_atom_feats = torch.as_tensor(rec.extra_atom_feats, dtype=torch.float32)
        if hasattr(rec, "has_xyz"): # (1,) bool/uint8
            d.has_xyz = torch.as_tensor(rec.has_xyz, dtype=torch.float32)
        if hasattr(rec, "dist"):
            # rec.dist is (L, L) (uint8) 
            d.hops = torch.as_tensor(rec.dist, dtype=torch.long).unsqueeze(0)  # (1, L, L)

        return d

In [ ]:
def make_loaders_for_task(
        task, 
        ids, 
        *, 
        batch_size=64, 
        seed=42,
        use_mixed_edges=True, 
        include_extra_atom_feats=True
        ):
    t = task2idx[task]
    tr_ids, va_ids = train_test_split(ids, test_size=0.2, random_state=seed)
    tr_ds = LMDBtoPyGSingleTask(
        tr_ids, 
        TRAIN_LMDB, 
        target_index=t, 
        use_mixed_edges=use_mixed_edges, 
        include_extra_atom_feats=include_extra_atom_feats
        )
    va_ds = LMDBtoPyGSingleTask(
        va_ids, 
        TRAIN_LMDB, 
        target_index=t,
        use_mixed_edges=use_mixed_edges,
        include_extra_atom_feats=include_extra_atom_feats
        )
    tr = GeoDataLoader(tr_ds, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True)
    va = GeoDataLoader(va_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    return tr, va

## Step 5: Define the Hybrid GNN Model

The final architecture uses both structural and cheminformatics data by combining GNN-learned graph embeddings with SMILES-derived RDKit descriptors. This Hybrid GNN model uses `smiles2graph` for graph construction and augments it with RDKit-based molecular features for improved prediction accuracy.

### Model Components:

* **AtomEncoder / BondEncoder**
  Transforms categorical atom and bond features (provided by OGB) into learnable embeddings using the encoders from `ogb.graphproppred.mol_encoder`. These provide a strong foundation for expressive graph learning.

* **GINEConv Layers (x2)**
  I use two stacked GINEConv layers (Graph Isomorphism Network with Edge features). These layers perform neighborhood aggregation based on edge attributes, allowing the model to capture localized chemical environments.

* **Global Mean Pooling**
  After message passing, node level embeddings are aggregated into a fixed size graph level representation using `global_mean_pool`.

* **Concatenation with RDKit Descriptors**
  The pooled GNN embedding is concatenated with external RDKit descriptors, which capture global molecular properties not easily inferred from graph data alone.

* **MLP Prediction Head**
  A multilayer perceptron processes the combined feature vector with ReLU activations, dropout regularization, and linear layers to predict the HOMO–LUMO gap.

In [ ]:
class DropPath(nn.Module):
    def __init__(self, drop_prob: float = 0.0):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x):
        if self.drop_prob == 0.0 or not self.training:
            return x
        keep = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = keep + torch.rand(shape, dtype=x.dtype, device=x.device)
        rand.floor_() # 0/1
        return x.div(keep) * rand

def _act(name: str):
    name = (name or "ReLU").lower()
    if name == "relu": return nn.ReLU()
    if name == "gelu": return nn.GELU()
    if name in ("swish", "silu"): return nn.SiLU()
    return nn.ReLU()

In [ ]:

class EdgeEncoderMixed(nn.Module):
    def __init__(self, emb_dim: int, cont_dim: int = 32, activation="GeLU"):
        super().__init__()
        act = _act(activation)
        # OGB bond categorical widths: type(5), stereo(6), conjugation(2)
        self.emb0 = nn.Embedding(5, emb_dim)
        self.emb1 = nn.Embedding(6, emb_dim)
        self.emb2 = nn.Embedding(2, emb_dim)
        self.mlp_cont = nn.Sequential(
            nn.Linear(cont_dim, emb_dim), act,
            nn.Linear(emb_dim, emb_dim),
            )

    def forward(self, edge_attr):
        # edge_attr: (E, 3+K)
        cat = edge_attr[:, :3].long()
        cont = edge_attr[:, 3:].float()
        e_cat  = self.emb0(cat[:,0]) + self.emb1(cat[:,1]) + self.emb2(cat[:,2])
        e_cont = self.mlp_cont(cont)
        return e_cat + e_cont


class ExtraAtomEncoder(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, activation="GeLU"):
        super().__init__()
        act = _act(activation)
        self.proj = nn.Sequential(
            nn.Linear(in_dim, out_dim), act,
            nn.Linear(out_dim, out_dim),
            )

    def forward(self, extra):
        return self.proj(extra) # (N, out_dim)


class GINEBlock_GNN(nn.Module):
    def __init__(self, dim, activation="GeLU", dropout=0.1, drop_path=0.0):
        super().__init__()
        act = _act(activation)

        self.norm1 = nn.LayerNorm(dim)
        self.conv = GINEConv(nn.Sequential(
            nn.Linear(dim, dim), act,
            nn.Linear(dim, dim),)
            )
        self.dropout1 = nn.Dropout(dropout)
        self.dp1 = DropPath(drop_path)

        self.norm2 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, 2*dim), act,
            nn.Dropout(dropout),
            nn.Linear(2*dim, dim))
        self.dropout2 = nn.Dropout(dropout)
        self.dp2 = DropPath(drop_path)

    def forward(self, x, edge_index, edge_emb):
        # pre-norm transformer style
        h = self.norm1(x)
        h = self.conv(h, edge_index, edge_emb)
        h = self.dropout1(h)
        x = x + self.dp1(h)
        h2 = self.norm2(x)
        h2 = self.ffn(h2)
        h2 = self.dropout2(h2)
        x = x + self.dp2(h2)
        return x

In [ ]:
class HybridGNNv2(nn.Module):
    def __init__(
            self,
            gnn_dim: int,
            rdkit_dim: int,
            hidden_dim: int,
            *,
            num_layers: int = 8,
            activation: str = "Swish",
            dropout: float = 0.2,
            drop_path_rate: float = 0.1,
            use_mixed_edges: bool = True,
            cont_dim: int = 32,
            use_extra_atom_feats: bool = True,
            extra_atom_dim: int = 5
            ):
        super().__init__()
        self.gnn_dim = gnn_dim
        self.rdkit_dim = rdkit_dim
        self.use_extra_atom_feats = use_extra_atom_feats

        # encoders
        self.atom_encoder = AtomEncoder(emb_dim=gnn_dim)
        if use_mixed_edges:
            self.edge_encoder = EdgeEncoderMixed(emb_dim=gnn_dim, cont_dim=cont_dim, activation=activation)
        else:
            self.edge_encoder = BondEncoder(emb_dim=gnn_dim)

        if use_extra_atom_feats:
            self.extra_atom = ExtraAtomEncoder(in_dim=extra_atom_dim, out_dim=gnn_dim, activation=activation)
            self.extra_gate = nn.Sequential(
                nn.Linear(2*gnn_dim, gnn_dim), _act(activation)
                )

        # backbone
        dpr = [drop_path_rate * i / max(1, num_layers - 1) for i in range(num_layers)]
        self.blocks = nn.ModuleList([
            GINEBlock_GNN(gnn_dim, activation=activation, dropout=dropout, drop_path=dpr[i])
            for i in range(num_layers)
        ])

        # pooling (concat of mean/max/attention)
        self.att_pool = GlobalAttention(
            gate_nn=nn.Sequential(
                nn.Linear(gnn_dim, gnn_dim // 2), _act(activation),
                nn.Linear(gnn_dim // 2, 1),
            )
        )
        pooled_dim = 3 * gnn_dim  # mean + max + attention
        # plus rdkit globals (+ optional has_xyz scalar)
        self.with_has_xyz = True
        head_in = pooled_dim + rdkit_dim + (1 if self.with_has_xyz else 0)

        self.head = nn.Sequential(
            nn.LayerNorm(head_in),
            nn.Linear(head_in, hidden_dim), _act(activation),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2), _act(activation),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1),
        )

    def forward(self, data):
        x = self.atom_encoder(data.x) # (N, D)

        if self.use_extra_atom_feats and hasattr(data, "extra_atom_feats"):
            xa = self.extra_atom(data.extra_atom_feats)  # (N, D)
            x = self.extra_gate(torch.cat([x, xa], dim=1))

        e = self.edge_encoder(data.edge_attr)

        for blk in self.blocks:
            x = blk(x, data.edge_index, e)

        # pool
        mean = global_mean_pool(x, data.batch)
        mmax = global_max_pool(x, data.batch)
        attn = self.att_pool(x, data.batch)
        g = torch.cat([mean, mmax, attn], dim=1)

        rd = data.rdkit_feats.view(g.size(0), -1)
        extras = [g, rd]

        if self.with_has_xyz and hasattr(data, "has_xyz"):
            # has_xyz collates to (B,1)
            extras.append(data.has_xyz.view(-1, 1).float())

        out = torch.cat(extras, dim=1)
        return self.head(out)

In [ ]:
def train_hybrid_gnn_sota(
        model: nn.Module,
        train_loader,
        val_loader,
        *,
        lr: float = 5e-4,
        optimizer: str = "AdamW",
        weight_decay: float = 1e-5,
        epochs: int = 120,
        warmup_epochs: int = 5,
        patience: int = 15,
        clip_norm: float = 1.0,
        amp: bool = True,
        loss_name: str = "mse",   # "mse" or "huber"
        save_dir: str=os.path.join(PROJECT_ROOT, "saved_models", "gnn"),
        tag: str = "model_sota",
        device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        ):
    
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)

    # optimizer
    opt_name = optimizer.lower()
    if opt_name == "rmsprop":
        opt = RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.0)
    else:
        opt = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # cosine schedule w/ warmup
    def lr_factor(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / max(1, warmup_epochs)
        t = (epoch - warmup_epochs) / max(1, (epochs - warmup_epochs))
        return 0.5 * (1 + math.cos(math.pi * t))
    scaler = GradScaler("cuda", enabled=amp)

    def loss_fn(pred, target):
        if loss_name.lower() == "huber":
            return F.huber_loss(pred, target, delta=1.0)
        return F.mse_loss(pred, target)

    @torch.no_grad()
    def eval_once(loader):
        model.eval()
        preds, trues = [], []
        for b in loader:
            b = b.to(device)
            p = model(b)
            preds.append(p.detach().cpu())
            trues.append(b.y.view(-1,1).cpu())
        preds = torch.cat(preds).numpy(); trues = torch.cat(trues).numpy()
        mae = np.mean(np.abs(preds - trues))
        rmse = float(np.sqrt(np.mean((preds - trues)**2)))
        r2 = float(1 - np.sum((preds - trues)**2) / np.sum((trues - trues.mean())**2))
        return mae, rmse, r2

    best_mae = float("inf")
    best = None
    best_path = os.path.join(save_dir, f"{tag}.pt")

    for ep in range(1, epochs+1):
        # schedule
        for g in opt.param_groups:
            g["lr"] = lr * lr_factor(ep-1)

        model.train()
        total, count = 0.0, 0
        for b in train_loader:
            b = b.to(device)
            with autocast("cuda", enabled=amp):
                pred = model(b)
                loss = loss_fn(pred, b.y.view(-1,1))

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            if clip_norm is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_norm)
            scaler.step(opt); scaler.update()

            total += loss.item() * b.num_graphs
            count += b.num_graphs

        tr_mse = total / max(1, count)
        mae, rmse, r2 = eval_once(val_loader)
        print(f"Epoch {ep:03d} | tr_MSE {tr_mse:.5f} | val_MAE {mae:.5f} | val_RMSE {rmse:.5f} | R2 {r2:.4f}")

        if mae < best_mae - 1e-6:
            best_mae = mae
            best = deepcopy(model.state_dict())
            torch.save(best, best_path)
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping.")
                break

    if best is not None:
        model.load_state_dict(best)
    else:
        model.load_state_dict(torch.load(best_path, map_location=device))

    final_mae, final_rmse, final_r2 = eval_once(val_loader)
    print(f"[{tag}] Best Val — MAE {final_mae:.6f} | RMSE {final_rmse:.6f} | R2 {final_r2:.4f}")
    return model, best_path, {"MAE": final_mae, "RMSE": final_rmse, "R2": final_r2}

In [ ]:
# Build loaders (now feeding mixed edges + extra atom feats)
train_loader_tg, val_loader_tg  = make_loaders_for_task("Tg", ids_tg, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_ffv,  val_loader_ffv  = make_loaders_for_task("FFV", ids_ffv, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_tc,  val_loader_tc  = make_loaders_for_task("Tc", ids_tc, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_den, val_loader_den = make_loaders_for_task("Density", ids_den, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_rg,  val_loader_rg  = make_loaders_for_task("Rg", ids_rg, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True)

b_tg = next(iter(train_loader_tg))
rd_dim = b_tg.rdkit_feats.shape[-1]        
print("rdkit_dim =", rd_dim)

# Tg 
model_tg = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_tg, ckpt_tg, metrics_tg = train_hybrid_gnn_sota(
    model_tg, train_loader_tg, val_loader_tg,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir=os.path.join(PROJECT_ROOT, "saved_models", "gnn", "gnn_tg_v2"), tag="hybridgnn_tg_v2"
)
# # FFV
model_ffv = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_ffv, ckpt_ffv, metrics_ffv = train_hybrid_gnn_sota(
    model_ffv, train_loader_ffv, val_loader_ffv,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir=os.path.join(PROJECT_ROOT, "saved_models", "gnn", "gnn_ffv_v2"), tag="hybridgnn_ffv_v2"
)

# Tc
model_tc = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_tc, ckpt_tc, metrics_tc = train_hybrid_gnn_sota(
    model_tc, train_loader_tc, val_loader_tc,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir=os.path.join(PROJECT_ROOT, "saved_models", "gnn", "gnn_tc_v2"), tag="hybridgnn_tc_v2"
)

# Density 
model_den = HybridGNNv2(
    gnn_dim=1024, rdkit_dim=rd_dim, hidden_dim=384,
    num_layers=12, activation="Swish", dropout=0.1, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)
model_den, ckpt_den, metrics_den = train_hybrid_gnn_sota(
    model_den, train_loader_den, val_loader_den,
    lr=5.956024201538505e-04, optimizer="AdamW", weight_decay=8.619671341229739e-06,
    epochs=200, warmup_epochs=8, patience=30,
    clip_norm=0.5, amp=True, loss_name="mse",
    save_dir=os.path.join(PROJECT_ROOT, "saved_models", "gnn", "gnn_density_v2"), tag="hybridgnn_density_v2"
)

# Rg 
model_rg = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)
model_rg, ckpt_rg, metrics_rg = train_hybrid_gnn_sota(
    model_rg, train_loader_rg, val_loader_rg,
    lr=5.6e-4, optimizer="RMSprop", weight_decay=9.0e-6,
    epochs=120, warmup_epochs=6, patience=20,
    clip_norm=0.5, amp=True, loss_name="huber",  # Huber often helps Rg
    save_dir=os.path.join(PROJECT_ROOT, "saved_models", "gnn", "gnn_rg_v2"), tag="hybridgnn_rg_v2"
)


| Model Type | Feature | MAE | RMSE | R2 |
|---|---|---|---|---|
| RF3D | Tg | 57.499522 | 74.296699 | 0.5867 |
| **GNN2** | **Tg** | **47.105114** | **61.480179** | **0.6040** |
| XGB  | Tg | 56.246650 | 70.835005 | 0.6224 |
| LGBM | Tg | 54.66728  | 68.213869 | 0.6499 |
| RF3D | Tc | 0.029678 | 0.002014 | 0.7331 |
| **GNN2** | **Tc** | **0.025115** | **0.041331** | **0.8000** |
| XGB  | Tc | 0.036923  | 0.002652 | 0.6486 |
| LGBM | Tc | 0.028768 | 0.002198 | 0.7088 |
| RF3D | Density | 0.037793 | 0.070932 | 0.7847 |
| GNN2 | Density | 0.031735 | 0.067845 | 0.7379 |
| **XGB**  | **Density** | **0.024079**  | **0.001544** | **0.9339** |
| LGBM | Density | 0.029702 | 0.002772 | 0.8814 |
| RF3D | FFV | 0.007621 | 0.017553 | 0.6605 |
| GNN2 | FFV | 0.013817 | 0.023902 | 0.4473 |
| **XGB**  | **FFV** | **0.005842**  | **0.000198** | **0.7816** |
| LGBM | FFV | 0.006494 | 0.000316 | 0.6515 |
| RF3D | Rg | 1.639309 | 2.493712 | 0.7296 |
| GNN2 | Rg | 2.083034 | 2.801481 | 0.6434 |
| XGB  | Rg | 1.581447 | 2.442744 | 0.7408 |
| **LGBM** | **Rg** | **1.557495** | **2.477095** | **0.7335** |








# Conclusions